<a href="https://colab.research.google.com/github/minhnguyen10/The-Battle-of-Neighborhoods-/blob/master/RECOMMENDATION_TO_OPEN_A_RESTAURANT_IN_TORONTO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json # library to handle JSON files
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup
import lxml
import html5lib
import requests
import pandas as pd
import numpy as np


### Part 1: Getting Canada Postal Code, Neighborhood, Borough

In [2]:
# url for CANADA Post Code from Wikipedia
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
res = requests.get(url)
soup = BeautifulSoup(res.content,"html.parser")

Canada_data = soup.find_all('table')[0] 
df = pd.read_html(str(Canada_data))[0]

In [3]:
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [4]:
df.columns = ['PostalCode','Borough','Neighborhood']

# keep only columns that include an assigned borough, and ignor cells with a borough that is Not assigned
toronto_data = df[df['Borough']!= 'Not assigned']
toronto_data = toronto_data.reset_index(drop=True)
toronto_data.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [5]:
toronto_data = toronto_data.groupby("PostalCode").agg(lambda x:','.join(set(x)))

In [6]:
na = toronto_data['Neighborhood'] == "Not assigned" #removed unassigned
toronto_data.loc[na, 'Neighborhood'] = toronto_data.loc[na, 'Borough']
toronto_data.reset_index(inplace=True)

In [7]:
#toronto_data
toronto_data.set_index(keys='PostalCode')
toronto_data.shape

(103, 3)

### Part 2: Import Latitude and Longitude of Postal Code

In [8]:
url = 'http://cocl.us/Geospatial_data'
df_gd = pd.read_csv(url)

In [9]:
df_gd.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [168]:
#Merge with the Neighborhood df
df_gd.rename(columns={'Postal Code':'PostalCode'},inplace=True)
df_gd.set_index(keys='PostalCode')
toronto_gd = pd.merge(toronto_data, df_gd, on='PostalCode' )
toronto_gd.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Part 3: Visualize Boroughs in Toronto on a map

In [15]:
toronto_boroughs= toronto_gd[toronto_gd['Borough'].str.contains('Toronto', na = False)].reset_index(drop=True)
toronto_boroughs.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [16]:
toronto_boroughs.shape

(39, 5)

In [17]:
address = 'Toronto, CAN'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6607002, -79.3850889.


In [19]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_boroughs['Latitude'], toronto_boroughs['Longitude'], 
                           toronto_boroughs['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Part 4: Get and Explore Venues in Toronto using Foursquare API

In [21]:
CLIENT_ID = 'CAGK1X4MRSIPEPDKOPEOAPGVJ33N35RJ221UVWQXLN5N2O4R' # your Foursquare ID
CLIENT_SECRET = 'FH1CJV5PGB14RC5ES14L1QSE0RU4OBXQONO3LG3MTPYRRP41' # your Foursquare Secret
VERSION = '20201222' # Foursquare API version
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CAGK1X4MRSIPEPDKOPEOAPGVJ33N35RJ221UVWQXLN5N2O4R
CLIENT_SECRET:FH1CJV5PGB14RC5ES14L1QSE0RU4OBXQONO3LG3MTPYRRP41


In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
toronto_venues = getNearbyVenues(names=toronto_boroughs['Neighborhood'],
                                   latitudes=toronto_boroughs['Latitude'],
                                   longitudes=toronto_boroughs['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West, Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High 

In [23]:
print(toronto_venues.shape)
toronto_venues.head()

(1599, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [24]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,57,57,57,57,57,57
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,59,59,59,59,59,59
Christie,16,16,16,16,16,16
Church and Wellesley,78,78,78,78,78,78
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,35,35,35,35,35,35


In [25]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 235 uniques categories.


In [127]:
Toronto_restaurants = toronto_venues[toronto_venues['Venue Category'].str.contains("Restaurant")]

In [128]:
print('There are {} unique cusines available in Toronto.'.format(len(Toronto_restaurants['Venue Category'].unique())))

There are 43 unique cusines available in Toronto.


### Part 5: Cluster and Analyze neighborhoods

In [129]:
# one hot encoding
toronto_onehot = pd.get_dummies(Toronto_restaurants[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = Toronto_restaurants['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,American Restaurant,Asian Restaurant,Belgian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Colombian Restaurant,Comfort Food Restaurant,Cuban Restaurant,Doner Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant,Greek Restaurant,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,New American Restaurant,Portuguese Restaurant,Ramen Restaurant,Restaurant,Seafood Restaurant,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Theme Restaurant,Tibetan Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
4,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
15,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [130]:
toronto_onehot.shape

(379, 44)

In [131]:
print('There are {} restaurants in Toronto with {} different style of cuisines.'.format(toronto_onehot.shape[0],(toronto_onehot.shape[1]-1)))

There are 379 restaurants in Toronto with 43 different style of cuisines.


In [132]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,American Restaurant,Asian Restaurant,Belgian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Colombian Restaurant,Comfort Food Restaurant,Cuban Restaurant,Doner Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Gluten-free Restaurant,Greek Restaurant,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,New American Restaurant,Portuguese Restaurant,Ramen Restaurant,Restaurant,Seafood Restaurant,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Theme Restaurant,Tibetan Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.083333,0.00,0.000000,0.083333,0.083333,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.000000,0.0000,0.000000,0.166667,0.166667,0.000000,0.0,0.083333,0.000000,0.000000,0.083333,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.000000,0.0000,0.000000,0.500000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.000000,0.0000,0.000000,0.500000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
3,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000,0.000000,0.062500,0.00,0.000000,0.000000,0.062500,0.187500,0.062500,0.062500,0.000000,0.000000,0.000000,0.0625,0.062500,0.000000,0.0,0.000000,0.0625,0.000000,0.000000,0.062500,0.062500,0.0,0.125000,0.000000,0.000000,0.062500,0.000000
4,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.000000,0.0000,0.000000,0.500000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
5,Church and Wellesley,0.037037,0.000000,0.000000,0.000000,0.000000,0.037037,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.037037,0.000000,0.074074,0.000000,0.000000,0.00,0.000000,0.000000,0.037037,0.037037,0.185185,0.037037,0.000000,0.074074,0.037037,0.0000,0.000000,0.000000,0.0,0.000000,0.0000,0.037037,0.111111,0.000000,0.185185,0.0,0.037037,0.037037,0.000000,0.000000,0.000000
6,"Commerce Court, Victoria Hotel",0.096774,0.064516,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.032258,0.000000,0.032258,0.00,0.032258,0.000000,0.000000,0.129032,0.096774,0.000000,0.032258,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.032258,0.0000,0.000000,0.225806,0.096774,0.000000,0.0,0.064516,0.000000,0.000000,0.064516,0.000000
7,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.100000,0.100000,0.200000,0.100000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0,0.000000,0.0000,0.00000

In [133]:
toronto_grouped.shape

(32, 44)

Let's print each neighborhood along with the top 5 most common venues

In [134]:
num_top_venues = 3

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                           venue  freq
0             Seafood Restaurant  0.17
1                     Restaurant  0.17
2  Vegetarian / Vegan Restaurant  0.08


----Brockton, Parkdale Village, Exhibition Place----
                   venue  freq
0     Italian Restaurant   0.5
1             Restaurant   0.5
2  Portuguese Restaurant   0.0


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                  venue  freq
0  Fast Food Restaurant   0.5
1            Restaurant   0.5
2   American Restaurant   0.0


----Central Bay Street----
                   venue  freq
0     Italian Restaurant  0.19
1        Thai Restaurant  0.12
2  Portuguese Restaurant  0.06


----Christie----
                   venue  freq
0     Italian Restaurant   0.5
1             Restaurant   0.5
2  Portuguese Restaurant   0.0


----Church and Wellesley----
                 venue  freq
0     Sushi Restaurant  0.19
1  Japanese Restaurant  0.19
2           Rest

Let's put that into a pandas dataframe

In [135]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [136]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Restaurant,Seafood Restaurant,Comfort Food Restaurant,Greek Restaurant,Indian Restaurant,Vegetarian / Vegan Restaurant,Japanese Restaurant,Eastern European Restaurant,French Restaurant,Thai Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Italian Restaurant,Restaurant,Doner Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant
2,"Business reply mail Processing Centre, South C...",Fast Food Restaurant,Restaurant,Vietnamese Restaurant,Doner Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Falafel Restaurant,Ethiopian Restaurant
3,Central Bay Street,Italian Restaurant,Thai Restaurant,Portuguese Restaurant,Falafel Restaurant,Indian Restaurant,Vegetarian / Vegan Restaurant,Japanese Restaurant,Korean Restaurant,Middle Eastern Restaurant,Modern European Restaurant
4,Christie,Italian Restaurant,Restaurant,Doner Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant


In [137]:
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

In [138]:
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import KMeans

import matplotlib.pyplot as plt
%matplotlib inline

import matplotlib.cm as cm
import matplotlib.colors as colors

In [148]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 4, 0, 2, 4, 2, 2, 2, 1, 2], dtype=int32)

In [151]:
neighborhoods_venues_sorted = neighborhoods_venues_sorted.drop(['Cluster Labels'], axis=1, errors='ignore')
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = Toronto_restaurants

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.fillna(0)
toronto_merged.head() # check the last columns!

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant,2,Greek Restaurant,Italian Restaurant,Restaurant,Caribbean Restaurant,Indian Restaurant,Japanese Restaurant,American Restaurant,Tibetan Restaurant,Seafood Restaurant,Cuban Restaurant
6,"The Danforth West, Riverdale",43.679557,-79.352188,Cafe Fiorentina,43.677743,-79.350115,Italian Restaurant,2,Greek Restaurant,Italian Restaurant,Restaurant,Caribbean Restaurant,Indian Restaurant,Japanese Restaurant,American Restaurant,Tibetan Restaurant,Seafood Restaurant,Cuban Restaurant
10,"The Danforth West, Riverdale",43.679557,-79.352188,Mezes,43.677962,-79.350196,Greek Restaurant,2,Greek Restaurant,Italian Restaurant,Restaurant,Caribbean Restaurant,Indian Restaurant,Japanese Restaurant,American Restaurant,Tibetan Restaurant,Seafood Restaurant,Cuban Restaurant
14,"The Danforth West, Riverdale",43.679557,-79.352188,7 Numbers,43.677062,-79.353934,Italian Restaurant,2,Greek Restaurant,Italian Restaurant,Restaurant,Caribbean Restaurant,Indian Restaurant,Japanese Restaurant,American Restaurant,Tibetan Restaurant,Seafood Restaurant,Cuban Restaurant
15,"The Danforth West, Riverdale",43.679557,-79.352188,Messini Authentic Gyros,43.677704,-79.350480,Greek Restaurant,2,Greek Restaurant,Italian Restaurant,Restaurant,Caribbean Restaurant,Indian Restaurant,Japanese Restaurant,American Restaurant,Tibetan Restaurant,Seafood Restaurant,Cuban Restaurant


In [152]:
toronto_merged['Cluster Labels'].value_counts()

2    350
0     21
4      4
1      3
3      1
Name: Cluster Labels, dtype: int64

In [153]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster, vc in zip(toronto_merged['Venue Latitude'], 
                                  toronto_merged['Venue Longitude'],
                                  toronto_merged['Venue'],
                                  toronto_merged['Cluster Labels'],
                                  toronto_merged['Venue Category'],):
    label = folium.Popup(str(poi) +' (' +str(vc) + ') ['+ str(cluster)+']', parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Examine Each cluster

In [154]:
c1 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[0] + list(range(8, toronto_merged.shape[1]))]]
c1.drop_duplicates()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
49,"India Bazaar, The Beaches West",Fast Food Restaurant,Italian Restaurant,Sushi Restaurant,Restaurant,Cuban Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Falafel Restaurant,Ethiopian Restaurant
119,"North Toronto West, Lawrence Park",Fast Food Restaurant,Mexican Restaurant,Chinese Restaurant,Restaurant,Vietnamese Restaurant,Doner Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Falafel Restaurant
317,"Regent Park, Harbourfront",French Restaurant,Mexican Restaurant,Restaurant,Vietnamese Restaurant,Doner Restaurant,German Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant
1483,"High Park, The Junction South",Thai Restaurant,Mexican Restaurant,Italian Restaurant,Cajun / Creole Restaurant,Fast Food Restaurant,Doner Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Falafel Restaurant
1588,"Business reply mail Processing Centre, South C...",Fast Food Restaurant,Restaurant,Vietnamese Restaurant,Doner Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Falafel Restaurant,Ethiopian Restaurant


In [155]:
c2 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 1,toronto_merged.columns[[0] + list(range(8, toronto_merged.shape[1]))]]
c2.drop_duplicates()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1060,"The Annex, North Midtown, Yorkville",Indian Restaurant,Middle Eastern Restaurant,Vietnamese Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant
1402,"Dufferin, Dovercourt Village",Middle Eastern Restaurant,Vietnamese Restaurant,Indian Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant


In [165]:
c3 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 2,toronto_merged.columns[[0] + list(range(8, toronto_merged.shape[1]))]]
c3.drop_duplicates()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,"The Danforth West, Riverdale",Greek Restaurant,Italian Restaurant,Restaurant,Caribbean Restaurant,Indian Restaurant,Japanese Restaurant,American Restaurant,Tibetan Restaurant,Seafood Restaurant,Cuban Restaurant
76,Studio District,American Restaurant,Seafood Restaurant,Comfort Food Restaurant,Latin American Restaurant,Middle Eastern Restaurant,Italian Restaurant,Thai Restaurant,Tibetan Restaurant,Filipino Restaurant,Fast Food Restaurant
136,Davisville,Italian Restaurant,Sushi Restaurant,Greek Restaurant,Indian Restaurant,Thai Restaurant,Seafood Restaurant,Restaurant,Japanese Restaurant,Doner Restaurant,Filipino Restaurant
173,"Summerhill West, Rathnelly, South Hill, Forest...",Vietnamese Restaurant,Sushi Restaurant,Restaurant,American Restaurant,Thai Restaurant,Cuban Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant
190,"St. James Town, Cabbagetown",Italian Restaurant,Restaurant,Indian Restaurant,Thai Restaurant,Taiwanese Restaurant,Japanese Restaurant,Caribbean Restaurant,Chinese Restaurant,Eastern European Restaurant,French Restaurant
235,Church and Wellesley,Sushi Restaurant,Japanese Restaurant,Restaurant,Fast Food Restaurant,Mediterranean Restaurant,Italian Restaurant,Mexican Restaurant,Caribbean Restaurant,Ethiopian Restaurant,Indian Restaurant
365,"Garden District, Ryerson",Japanese Restaurant,Middle Eastern Restaurant,Italian Restaurant,Ramen Restaurant,Fast Food Restaurant,Chinese Restaurant,Ethiopian Restaurant,Mexican Restaurant,Modern European Restaurant,Vietnamese Restaurant
460,St. James Town,American Restaurant,Moroccan Restaurant,Restaurant,Japanese Restaurant,Asian Restaurant,Belgian Restaurant,Comfort Food Restaurant,French Restaurant,German Restaurant,Vegetarian / Vegan Restaurant
539,Berczy Park,Restaurant,Seafood Restaurant,Comfort Food Restaurant,Greek Restaurant,Indian Restaurant,Vegetarian / Vegan Restaurant,Japanese Restaurant,Eastern European Restaurant,French Restaurant,Thai Restaurant
598,Central Bay Street,Italian Restaurant,Thai Restaurant,Portuguese Restaurant,Falafel Restaurant,Indian Restaurant,Vegetarian / Vegan Restaurant,Japanese Restaurant,Korean Restaurant,Middle Eastern Restaurant,Modern European Restaurant


In [166]:
c4 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 3,toronto_merged.columns[[0] + list(range(8, toronto_merged.shape[1]))]]
c4.drop_duplicates()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1057,"Forest Hill North & West, Forest Hill Road Park",Sushi Restaurant,Vietnamese Restaurant,Doner Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant


In [167]:
c5 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 4,toronto_merged.columns[[0] + list(range(8, toronto_merged.shape[1]))]]
c5.drop_duplicates()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1388,Christie,Italian Restaurant,Restaurant,Doner Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant
1461,"Brockton, Parkdale Village, Exhibition Place",Italian Restaurant,Restaurant,Doner Restaurant,Gluten-free Restaurant,German Restaurant,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant
